# Importe De Librerías

In [246]:
import pandas as pd
import numpy as np
import xgboost as xgb
import sklearn
import pickle
import matplotlib.pyplot as plt

from sklearn.preprocessing import scale, MinMaxScaler
from sklearn.model_selection import StratifiedKFold, KFold,RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn import metrics

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import AdaBoostRegressor
from six import StringIO
import pydotplus
from IPython.display import Image  
from sklearn.tree import export_graphviz

import warnings
warnings.filterwarnings('always')
from sklearn.linear_model import LinearRegression
from sklearn import tree, preprocessing, model_selection, ensemble

# Regresión

### Lectura de archivos

In [247]:
ds_regresion = pd.read_csv("properati_argentina_train.csv")
test = pd.read_csv("properati_argentina_test.csv")

ds_regresion_reducido = pd.read_csv("properati_argentina_reducido_train.csv")
test_reducido = pd.read_csv("properati_argentina_reducido_test.csv")

variables_a_eliminar = ["Unnamed: 0"]

ds_regresion.drop(columns=variables_a_eliminar, inplace=True)
test.drop(columns = variables_a_eliminar, inplace=True)

ds_regresion_reducido.drop(columns=variables_a_eliminar, inplace=True)
test_reducido.drop(columns = variables_a_eliminar, inplace=True)

## KNN

### Preparo los datasets


In [248]:
ds_train_knn = ds_regresion.copy()
ds_test_knn = test.copy()

ds_train_reducido_knn = ds_regresion_reducido.copy()
ds_test_reducido_knn = test_reducido.copy()

#Eliminamos las filas que contengan algún NaN, ya que sino fallan los calculos de las metricas

ds_train_knn = ds_train_knn.dropna()
ds_test_knn = ds_test_knn.dropna()

ds_train_reducido_knn = ds_train_reducido_knn.dropna()
ds_test_reducido_knn = ds_test_reducido_knn.dropna()

#### Primero entrenamos el modelo con el dataset no reducido

#### Z-Score - Ingeniería De Características

In [249]:
#Escalamos las variables para que las variables con valores más grandes, como 'property surface total', no tengan más peso que las más pequeñas. 
#Esto podría perjudicar las predicciones del modelo. Utilizamos 
standard_scaler = StandardScaler()

ds_train_knn["property_surface_covered"] = standard_scaler.fit_transform(ds_train_knn["property_surface_covered"].to_frame())
ds_train_knn["property_surface_total"] = standard_scaler.fit_transform(ds_train_knn["property_surface_total"].to_frame())
ds_train_knn["longitud"] = standard_scaler.fit_transform(ds_train_knn["longitud"].to_frame())
ds_train_knn["latitud"] = standard_scaler.fit_transform(ds_train_knn["latitud"].to_frame())

ds_test_knn["property_surface_covered"] = standard_scaler.fit_transform(ds_test_knn["property_surface_covered"].to_frame())
ds_test_knn["property_surface_total"] = standard_scaler.fit_transform(ds_test_knn["property_surface_total"].to_frame())
ds_test_knn["longitud"] = standard_scaler.fit_transform(ds_test_knn["longitud"].to_frame())
ds_test_knn["latitud"] = standard_scaler.fit_transform(ds_test_knn["latitud"].to_frame())


ds_train_knn

,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price
0,-0.202830,-0.825543,Paternal,Departamento,2.0,1.0,-0.357417,-0.422278,80000.0
1,0.628299,0.367785,Palermo,Departamento,2.0,1.0,-0.614211,-0.774079,79900.0
2,1.723005,-0.358624,Belgrano,Departamento,1.0,1.0,-0.756875,-1.151008,69000.0
3,-1.369360,-0.689731,Flores,Departamento,5.0,3.0,-0.043558,0.231066,150000.0
4,1.807274,-0.642786,Nuñez,Departamento,2.0,1.0,-0.457282,-0.799207,85000.0
...,...,...,...,...,...,...,...,...,...
59846,-1.320864,1.811781,Barracas,Departamento,1.0,1.0,-0.599945,-0.849465,70000.0
59847,1.493074,-0.665072,Belgrano,Departamento,3.0,2.0,-0.043558,0.231066,158000.0
59848,-1.232492,0.661945,Boedo,Departamento,3.0,2.0,0.313101,0.356709,175000.0
59849,0.351606,-0.353310,Chacarita,Departamento,2.0,1.0,-0.243286,-0.170992,122000.0


#### One Hot Encoding - Ingeniería De Características

In [250]:
#Realizamos One Hot Encoding para las variables categoricas del dataset

ds_train_knn = pd.get_dummies(ds_train_knn, columns=["property_type"], drop_first=True)
ds_test_knn = pd.get_dummies(ds_test_knn, columns=["property_type"], drop_first=True)

#### Eliminamos variables

In [251]:
#Eliminamos la variable place_l3, ya que realizar One Hot Encoding generaría una nueva variable por cada barrio, 
#lo cual agrandaría demasiado la dimensionalidad del dataset, perjudicanod la performance del modelo.

variables_eliminadas=["place_l3"]
ds_train_knn.drop(variables_eliminadas, axis='columns', inplace=True)
ds_test_knn.drop(variables_eliminadas, axis='columns', inplace=True)

In [252]:
#Estado actual del dataset 
ds_train_knn


,latitud,longitud,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_type_Departamento,property_type_PH
0,-0.202830,-0.825543,2.0,1.0,-0.357417,-0.422278,80000.0,1,0
1,0.628299,0.367785,2.0,1.0,-0.614211,-0.774079,79900.0,1,0
2,1.723005,-0.358624,1.0,1.0,-0.756875,-1.151008,69000.0,1,0
3,-1.369360,-0.689731,5.0,3.0,-0.043558,0.231066,150000.0,1,0
4,1.807274,-0.642786,2.0,1.0,-0.457282,-0.799207,85000.0,1,0
...,...,...,...,...,...,...,...,...,...
59846,-1.320864,1.811781,1.0,1.0,-0.599945,-0.849465,70000.0,1,0
59847,1.493074,-0.665072,3.0,2.0,-0.043558,0.231066,158000.0,1,0
59848,-1.232492,0.661945,3.0,2.0,0.313101,0.356709,175000.0,1,0
59849,0.351606,-0.353310,2.0,1.0,-0.243286,-0.170992,122000.0,1,0


In [253]:
#Estado actual del dataset
ds_test_knn

,latitud,longitud,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_type_Departamento,property_type_PH
0,-0.094443,-1.607297,1.0,1.0,-0.816679,-0.965590,85000.0,1,0
1,0.804779,0.492298,2.0,1.0,1.487639,0.094394,520000.0,1,0
2,0.366765,1.085025,2.0,1.0,-0.713808,-0.637500,87000.0,1,0
3,0.703809,0.878381,4.0,3.0,1.076153,1.305804,690000.0,1,0
4,1.335705,-0.665890,1.0,1.0,-0.919550,-0.940352,95000.0,1,0
...,...,...,...,...,...,...,...,...,...
16257,-0.865175,-0.602410,3.0,2.0,-0.034857,-0.031795,152000.0,1,0
16258,0.882737,-1.129102,2.0,1.0,-0.652085,-0.637500,109000.0,1,0
16259,0.320362,0.862865,2.0,1.0,-0.631511,-0.662737,194000.0,1,0
16260,-0.958197,1.050268,3.0,2.0,0.047440,0.144869,142000.0,1,0


#### Optimización de parametros con Random Search

In [254]:
x_train = ds_train_knn.drop(columns=["property_price"])
y_train = ds_train_knn["property_price"]

Decidimos usar Random Search para la búsqueda de hiperparametros. Si bien Grid Search permite buscar más combinaciones, su timepo de ejecución puede ser demasiado alto, y no nos parece que se justifique su utilización. Random Search es capaz de darnos hiperparametros que generan buenas métricas con menos timepo de ejecución. Usamos 5 K-Folds

In [255]:
#Grilla de Parámetros
params_grid={ 'n_neighbors':range(1,30), 
              'weights':['distance','uniform'],
              'algorithm':['ball_tree', 'kd_tree', 'brute'],
              'metric':['euclidean','manhattan','chebyshev', 'minkowski'],
              'leaf_size': list(range(0, 50)),
             }

#Metrica que quiero optimizar MSE
scorer_fn = make_scorer(sklearn.metrics.mean_squared_error)

#Clasificador KNN
knn=KNeighborsRegressor()

#Random Search con 10 Folds y 10 iteraciones
rand = RandomizedSearchCV(knn, params_grid, cv=5, scoring=scorer_fn, n_iter=10, random_state=5)

rand.fit(x_train, y_train)

RandomizedSearchCV(cv=8, estimator=KNeighborsRegressor(),
                   param_distributions={'algorithm': ['ball_tree', 'kd_tree',
                                                      'brute'],
                                        'leaf_size': [0, 1, 2, 3, 4, 5, 6, 7, 8,
                                                      9, 10, 11, 12, 13, 14, 15,
                                                      16, 17, 18, 19, 20, 21,
                                                      22, 23, 24, 25, 26, 27,
                                                      28, 29, ...],
                                        'metric': ['euclidean', 'manhattan',
                                                   'chebyshev', 'minkowski'],
                                        'n_neighbors': range(1, 30),
                                        'weights': ['distance', 'uniform']},
                   random_state=5, scoring=make_scorer(mean_squared_error))

In [256]:
#Mejores hiperparametros
print(rand.best_params_)

#Mejor métrica
mse = rand.best_score_
print("RMSE en datos de entrnamiento: " + str(np.sqrt(mse)))

{'weights': 'uniform', 'n_neighbors': 24, 'metric': 'euclidean', 'leaf_size': 38, 'algorithm': 'kd_tree'}
RMSE en datos de entrnamiento: 57116.57179041209


Entrenamos el modelo

In [257]:
#Mejor estimador
best_knn=rand.best_estimator_

x_test_knn = ds_test_knn.drop(columns=["property_price"])
y_test_knn = ds_test_knn["property_price"]

y_pred_knn = best_knn.predict(x_test_knn)

In [258]:
#Metricas para evaluar modelos
from sklearn import metrics

#Root Mean Square Error
rmse = metrics.mean_squared_error(
        y_true  = y_test_knn,
        y_pred  = y_pred_knn,
        squared = False
       )

print(f"El error (rmse) de test es: {rmse}")

El error (rmse) de test es: 58122.91231534372


Buscamos los mejores hiperparametros que optimizen el MSE, ya que es la medida que nos interesa a al hora de hacer regresion. Queremos calcular el error medio de las predicciones, cuanto menor sea el error, mejor será la performance del modelo.

La metrica que utlizamos para evaluar el modelo es el RMSE. La ventaja de RMSE sobre MSE es que los valores de la métrica se corresponden con las unidades de la variable target que se predice. Así es más facil de dimensionar y entender cuanto es el error. 

La performance del modelo en el conjunto de entrenamiento fue muy similar a la de testeo. Esto significa que no hay overfitting, de haberlo la performance en el conjunto de entrenamiento sería mucho mejor que en el de testeo. Tampoco creemos que haya underfitting. Un RMSE de aproximadamente 60.000 tanto en conjunto de test como en el de entrenamiento, si bien no creemos que sea bajo, no lo consideramos suficiente como para afirmar que el modelo sufre de underfitting. Dado esto consideramos que es un modelo robusto.

#### Ahora entrenamos el modelo con el dataset reducido

In [259]:
#Estado del dataset reducido
ds_train_reducido_knn.head()

,cp_1,cp_2,cp_3,cp_4,cp_5,cp_6,target
0,-1.203931,-0.275293,0.413176,0.281218,-0.653813,0.322816,80000.0
1,-1.556395,0.103435,0.233243,-0.015272,0.565592,-0.188810,79900.0
2,-2.230063,-0.194028,1.526780,-0.091470,0.809760,-0.160262,69000.0
3,1.678872,0.694012,-0.608398,0.044571,-2.015136,-0.732341,150000.0
4,-1.511679,0.021077,1.780005,-0.186017,0.574209,-0.148672,85000.0


In [260]:
#Estado del dataset reducido
ds_test_reducido_knn.head()

,cp_1,cp_2,cp_3,cp_4,cp_5,cp_6,target
0,-1.203931,-0.275293,0.413176,0.281218,-0.653813,0.322816,85000.0
1,-1.556395,0.103435,0.233243,-0.015272,0.565592,-0.188810,520000.0
2,-2.230063,-0.194028,1.526780,-0.091470,0.809760,-0.160262,87000.0
3,1.678872,0.694012,-0.608398,0.044571,-2.015136,-0.732341,690000.0
4,-1.511679,0.021077,1.780005,-0.186017,0.574209,-0.148672,95000.0


#### Optimizacion de parametros con Random search

In [261]:
x_train = ds_train_reducido_knn.drop(columns=["target"])
y_train = ds_train_reducido_knn["target"]

Nuevamente, y por las razones ya mencionadas más arriba, utilizamos Random Search y no Grid Search. Usamos 5 K-Folds

In [262]:
#Grilla de Parámetros
params_grid={ 'n_neighbors':range(1,30), 
              'weights':['distance','uniform'],
              'algorithm':['ball_tree', 'kd_tree', 'brute'],
              'metric':['euclidean','manhattan','chebyshev', 'minkowski'],
              'leaf_size': list(range(0, 50))
             }

#Metrica que quiero optimizar MSE
scorer_fn = make_scorer(sklearn.metrics.mean_squared_error)

#Clasificador KNN
knn=KNeighborsRegressor()

#Random Search con 10 Folds y 10 iteraciones
rand = RandomizedSearchCV(knn, params_grid, cv=5, scoring=scorer_fn, n_iter=10, random_state=5)

rand.fit(x_train, y_train)

RandomizedSearchCV(cv=8, estimator=KNeighborsRegressor(),
                   param_distributions={'algorithm': ['ball_tree', 'kd_tree',
                                                      'brute'],
                                        'leaf_size': [0, 1, 2, 3, 4, 5, 6, 7, 8,
                                                      9, 10, 11, 12, 13, 14, 15,
                                                      16, 17, 18, 19, 20, 21,
                                                      22, 23, 24, 25, 26, 27,
                                                      28, 29, ...],
                                        'metric': ['euclidean', 'manhattan',
                                                   'chebyshev', 'minkowski'],
                                        'n_neighbors': range(1, 30),
                                        'weights': ['distance', 'uniform']},
                   random_state=5, scoring=make_scorer(mean_squared_error))

In [263]:
#Mejores hiperparametros
print(rand.best_params_)

#Mejor métrica
mse = rand.best_score_
print("RMSE en datos de entrnamiento: " + str(np.sqrt(mse)))


{'weights': 'uniform', 'n_neighbors': 24, 'metric': 'euclidean', 'leaf_size': 38, 'algorithm': 'kd_tree'}
RMSE en datos de entrnamiento: 56221.40305216622


Entrenamos el modelo

In [264]:
#Mejor estimador
best_knn=rand.best_estimator_

x_test_reducido_knn = ds_test_reducido_knn.drop(columns=["target"])
y_test_reducido_knn = ds_test_reducido_knn["target"]

y_pred_knn = best_knn.predict(x_test_reducido_knn)

In [265]:
#Metricas para evaluar modelos
from sklearn import metrics

#Root Mean Square Error
rmse = metrics.mean_squared_error(
        y_true  = y_test_reducido_knn,
        y_pred  = y_pred_knn,
        squared = False
       )

print(f"El error (rmse) de test es: {rmse}")

El error (rmse) de test es: 189082.8080318387


Nuevamente, y por las mismas razones, buscamos los hiperparametros que optimizen MSE. A la hora de analizar la performance, volvemos a utilizar RMSE por la razones mencionadas previamente

Persistimos el modelo

In [266]:
#filename = "knn.sav"
#pickle.dump(best_knn, open(filename, 'wb'))

## XGBoost

In [271]:
ds_prop_XGBoost_train = ds_regresion.copy()
ds_prop_XGBoost_test = test.copy()

ds_prop_XGBoost_reducido_train = ds_regresion_reducido.copy()
ds_prop_XGBoost_reducido_test = test_reducido.copy()

ds_prop_XGBoost_train = ds_prop_XGBoost_train.dropna()
ds_prop_XGBoost_test = ds_prop_XGBoost_test.dropna()

ds_prop_XGBoost_reducido_train = ds_prop_XGBoost_reducido_train.dropna()
ds_prop_XGBoost_reducido_test = ds_prop_XGBoost_reducido_test.dropna()

#### Primero entrenamos el modelo con el dataset sin reducir

#### Ingeniería De Caraterísticas

Z-Score - Ingeniería De Características

In [272]:
#Escalamos las variables para que no tengan mayor peso
standard_scaler = StandardScaler()

ds_prop_XGBoost_train["property_surface_covered"] = standard_scaler.fit_transform(ds_prop_XGBoost_train["property_surface_covered"].to_frame())
ds_prop_XGBoost_train["property_surface_total"] = standard_scaler.fit_transform(ds_prop_XGBoost_train["property_surface_total"].to_frame())
ds_prop_XGBoost_train["longitud"] = standard_scaler.fit_transform(ds_prop_XGBoost_train["longitud"].to_frame())
ds_prop_XGBoost_train["latitud"] = standard_scaler.fit_transform(ds_prop_XGBoost_train["latitud"].to_frame())
#ds_prop_XGBoost_train["property_rooms"] = standard_scaler.fit_transform(ds_prop_XGBoost_train["property_rooms"].to_frame())

ds_prop_XGBoost_test["property_surface_covered"] = standard_scaler.fit_transform(ds_prop_XGBoost_test["property_surface_covered"].to_frame())
ds_prop_XGBoost_test["property_surface_total"] = standard_scaler.fit_transform(ds_prop_XGBoost_test["property_surface_total"].to_frame())
ds_prop_XGBoost_test["longitud"] = standard_scaler.fit_transform(ds_prop_XGBoost_test["longitud"].to_frame())
ds_prop_XGBoost_test["latitud"] = standard_scaler.fit_transform(ds_prop_XGBoost_test["latitud"].to_frame())
#ds_prop_XGBoost_test["property_rooms"] = standard_scaler.fit_transform(ds_prop_XGBoost_test["property_rooms"].to_frame())



One Hot Encoding - Ingeniería De Características

In [273]:
variables_reemplazadas = ["property_type"]
ds_prop_XGBoost_train = pd.get_dummies(ds_prop_XGBoost_train, columns=variables_reemplazadas, drop_first=True)
ds_prop_XGBoost_test = pd.get_dummies(ds_prop_XGBoost_test, columns=variables_reemplazadas, drop_first=True)

Eliminamos Variables

In [274]:
variables_eliminadas=["place_l3"]
ds_prop_XGBoost_train.drop(variables_eliminadas, axis='columns', inplace=True)
ds_prop_XGBoost_test.drop(variables_eliminadas, axis='columns', inplace=True)

#### Optimización de parametros con Random Search

In [275]:
features = list(ds_prop_XGBoost_train.columns.values)
features.pop(features.index("property_price"))

target = ["property_price"]

x_train = ds_prop_XGBoost_train[features]
x_test = ds_prop_XGBoost_test[features]

y_train = ds_prop_XGBoost_train[target]
y_test = ds_prop_XGBoost_test[target]

Nuevamente, por las razones ya explicadas, buscamos los mejores hiperparametros usando Random CV. Usamos 8 K-Folds

In [276]:
#KFOLD CV Random Search para buscar el mejor arbol (los mejores atributos, hiperparametros,etc)

#Cantidad de combinaciones que quiero porbar
n=10

#Conjunto de parámetros que quiero usar
params_grid = {'criterion':['gini','entropy'],                  #Luego de probar con varias combinaciones de parametros se
               'ccp_alpha':np.linspace(0,0.5,15),               #llegó a la conclusión de que Random Search encuentra los
               'max_depth':list(range(3,4)),                    #hiperparametros más adecuados con esta grilla 
               'random_state':list(range(0,6)),                 #de parametros
               'gamma':list(range(31,32)),
               'min_child_weight':list(range(17,18)),
               'colsample_bytree':np.linspace(0.75,0.8,15)}
                
#Cantidad de splits para el Cross Validation
folds=8

#Regresor
xgb_model_rd_search = xgb.XGBRegressor()

#Metrica que quiero optimizar MSE
scorer_fn = make_scorer(sklearn.metrics.mean_squared_error)

#Random Search Cross Validation
randomcv = RandomizedSearchCV(estimator=xgb_model_rd_search,
                              param_distributions = params_grid,
                              scoring=scorer_fn,
                              n_iter=n, cv=folds, random_state=5) 


#Busco los hiperparamtros que optimizan MSE
randomcv.fit(x_train,y_train)

[17:13:48] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "ccp_alpha", "criterion" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[17:13:49] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "ccp_alpha", "criterion" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[17:13:49] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "ccp_alpha", "criterio

RandomizedSearchCV(cv=8,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, gamma=None,
                                          gpu_id=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=None, max_bin=None,
                                          m...
       0.35714286, 0.39285714, 0.42857143, 0.46428571, 0.5       ]),
                                        'colsample_bytree': array([0.75      , 0.75357143, 0.75714286, 0.76071429, 0.76428571,
       0.76785714, 0.77142857, 0.775     , 0.77857143, 0.78214286,
       0.78571429, 0.78928571, 0.79285714, 0.79642857, 0.8       ]),
                                        'criterion': ['gini', 'entropy'],
                                        'gamma': [31], 'max_depth': [3],
                                        'min_child_weight': [17],
                                        'random_state': [0, 1, 2, 3, 4, 5]},
                   random_state=5, scoring=make_scorer(mean_squared_error))

In [277]:
#Mejores hiperparametros
print(randomcv.best_params_)

#Mejor métrica
mse = randomcv.best_score_
print("RMSE en datos de entrnamiento: " + str(np.sqrt(mse)))

{'random_state': 4, 'min_child_weight': 17, 'max_depth': 3, 'gamma': 31, 'criterion': 'entropy', 'colsample_bytree': 0.775, 'ccp_alpha': 0.42857142857142855}
RMSE en datos de entrnamiento: 55321.007109548824


Entrenamos el modelo

In [278]:
xgb_model = xgb.XGBRegressor().set_params(**randomcv.best_params_)
xgb_model.fit(x_train, y_train)

[17:15:10] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "ccp_alpha", "criterion" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             ccp_alpha=0.42857142857142855, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.775, criterion='entropy',
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=31, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=17,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=4, ...)

In [279]:
from sklearn.metrics import mean_squared_error

y_pred = xgb_model.predict(x_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
print("RMSE en datos de test: " + str(rmse))

RMSE en datos de test: 59273.0969600755


Volvemos a buscar los hiperparametros que optimizen MSE. Al ser hacer regresión, la métrica más conveniente es MSE, queremos el error cuadrático medio 
Nuevamente medimos que tan precisas fueron las predicciones usando RMSE. La ventaja de usar RMSE en vez de MSE es que el valor de RMSE está en las mismas unidades que la variable target.

La performance del conjunto de test y de entrenamiento fue similar, lo cual es bueno. El hecho de que las performances sean sean similares nos indica que no hay overfitting. Las metricas en ambos conjuntos no son altas, por lo tanto no consideramos que haya underfitting

#### Ahora probamos el modelo con el dataset reducido

In [280]:
x_train = ds_prop_XGBoost_reducido_train.drop(columns=["target"])
y_train = ds_prop_XGBoost_reducido_train["target"]

x_test = ds_prop_XGBoost_reducido_test.drop(columns=["target"])
y_test = ds_prop_XGBoost_reducido_test["target"]

Volvemos a usar Random Search para buscar los hiperparametros por las razones explicadas previamente. Usamos 8 K-Folds

In [281]:
#KFOLD CV Random Search para buscar el mejor arbol (los mejores atributos, hiperparametros,etc)

#Cantidad de combinaciones que quiero porbar
n=10

#Conjunto de parámetros que quiero usar
params_grid = {'criterion':['gini','entropy'],                  #Luego de probar con varias combinaciones de parametros se
               'ccp_alpha':np.linspace(0,0.5,15),               #llegó a la conclusión de que Random Search encuentra los
               'max_depth':list(range(3,4)),                    #hiperparametros más adecuados con esta grilla 
               'random_state':list(range(0,6)),                 #de parametros
               'gamma':list(range(31,32)),
               'min_child_weight':list(range(17,18)),
               'colsample_bytree':np.linspace(0.75,0.8,15)}
                
#Cantidad de splits para el Cross Validation
folds=8

#Regresor
xgb_model_rd_search = xgb.XGBRegressor()

#Metrica que quiero optimizar MSE
scorer_fn = make_scorer(sklearn.metrics.mean_squared_error)

#Random Search Cross Validation
randomcv = RandomizedSearchCV(estimator=xgb_model_rd_search,
                              param_distributions = params_grid,
                              scoring=scorer_fn,
                              n_iter=n, cv=folds, random_state=5) 


#Busco los hiperparamtros que optimizan MSE
randomcv.fit(x_train,y_train)

[17:15:12] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "ccp_alpha", "criterion" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[17:15:13] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "ccp_alpha", "criterion" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[17:15:15] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "ccp_alpha", "criterio

RandomizedSearchCV(cv=8,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, gamma=None,
                                          gpu_id=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=None, max_bin=None,
                                          m...
       0.35714286, 0.39285714, 0.42857143, 0.46428571, 0.5       ]),
                                        'colsample_bytree': array([0.75      , 0.75357143, 0.75714286, 0.76071429, 0.76428571,
       0.76785714, 0.77142857, 0.775     , 0.77857143, 0.78214286,
       0.78571429, 0.78928571, 0.79285714, 0.79642857, 0.8       ]),
                                        'criterion': ['gini', 'entropy'],
                                        'gamma': [31], 'max_depth': [3],
                                        'min_child_weight': [17],
                                        'random_state': [0, 1, 2, 3, 4, 5]},
                   random_state=5, scoring=make_scorer(mean_squared_error))

In [282]:
#Mejores hiperparametros
print(randomcv.best_params_)

#Mejor métrica
mse = randomcv.best_score_
print("RMSE en datos de entrnamiento: " + str(np.sqrt(mse)))

{'random_state': 0, 'min_child_weight': 17, 'max_depth': 3, 'gamma': 31, 'criterion': 'gini', 'colsample_bytree': 0.7892857142857144, 'ccp_alpha': 0.17857142857142855}
RMSE en datos de entrnamiento: 59332.498392953465


Entrenamos el modelo

In [283]:
xgb_model = xgb.XGBRegressor().set_params(**randomcv.best_params_)
xgb_model.fit(x_train, y_train)

[17:17:00] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "ccp_alpha", "criterion" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             ccp_alpha=0.17857142857142855, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7892857142857144,
             criterion='gini', early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, gamma=31, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_to_onehot=4, max_delta_step=0, max_depth=3, max_leaves=0,
             min_child_weight=17, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, predictor='auto',
             random_state=0, ...)

Medimos que tan precisas fueron las predicciones usando RMSE. La ventaja de usar RMSE en vez de MSE es que el valor de RMSE está en las mismas unidades que la variable target. De esta forma es más fácil dimensionar que tan preciso es el modelo creado

In [284]:
from sklearn.metrics import mean_squared_error

y_pred = xgb_model.predict(x_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
print("RMSE en datos de test: " + str(rmse))

RMSE en datos de test: 187584.85333177503


Vemos que la performance de entrenamiento y la del conjunto de evaluación es bastante similar. Esto significa que no hay overfitting, es decir, el modelo no se aprende de 'memoria' los datos de entrenamiento y no pierde mucha precision al realizar las predicciones en el conjunto de evaluación. 

In [285]:
#Atributos considerados y su importancia
sorted(list(zip(x_train.columns.to_list(), xgb_model.feature_importances_)), key=lambda x: -x[1])

[('cp_2', 0.45453882),
 ('cp_1', 0.19503015),
 ('cp_6', 0.13273698),
 ('cp_4', 0.11194253),
 ('cp_5', 0.054574847),
 ('cp_3', 0.051176704)]

Por ultimo, persistimos el modelo

In [286]:
#filename = "xgb_model.sav"
#pickle.dump(xgb_model, open(filename, 'wb'))

## GradientBoost

In [306]:
ds_prop_GBoost_train = ds_regresion.copy()
ds_prop_GBoost_test = test.copy()

ds_prop_GBoost_reducido_train = ds_regresion_reducido.copy()
ds_prop_GBoost_reducido_test = test_reducido.copy()

ds_prop_GBoost_train = ds_prop_GBoost_train.dropna()
ds_prop_GBoost_test = ds_prop_GBoost_test.dropna()

ds_prop_GBoost_reducido_train = ds_prop_GBoost_reducido_train.dropna()
ds_prop_GBoost_reducido_test = ds_prop_GBoost_reducido_test.dropna()

#### Primero entrenamos el modelo con el dataset sin reducir

#### Ingenieria De Caracteristicas

Z-Score - Ingeniera De Caracteristicas

In [307]:
#One Hot Encoding para variables categoricas

variables_reemplazadas = ["property_type"]
ds_prop_GBoost_train = pd.get_dummies(ds_prop_GBoost_train, columns=variables_reemplazadas, drop_first=True)
ds_prop_GBoost_test = pd.get_dummies(ds_prop_GBoost_test, columns=variables_reemplazadas, drop_first=True)

#Eliminamos place_l3 ya que hacer One Hot Encoding agrandaría demasiado el dataset 
variables_eliminadas=["place_l3"]
ds_prop_GBoost_train.drop(variables_eliminadas, axis='columns', inplace=True)
ds_prop_GBoost_test.drop(variables_eliminadas, axis='columns', inplace=True)


#Escalamos las variables para que no tengan mayor peso
scaler = MinMaxScaler()

#ds_prop_GBoost_train["property_surface_covered"] = scaler.fit_transform(ds_prop_GBoost_train["property_surface_covered"].to_frame())
#ds_prop_GBoost_train["property_surface_total"] = scaler.fit_transform(ds_prop_GBoost_train["property_surface_total"].to_frame())
#ds_prop_GBoost_train["longitud"] = scaler.fit_transform(ds_prop_GBoost_train["longitud"].to_frame())
#ds_prop_GBoost_train["latitud"] = scaler.fit_transform(ds_prop_GBoost_train["latitud"].to_frame())

#ds_prop_GBoost_test["property_surface_covered"] = scaler.fit_transform(ds_prop_GBoost_test["property_surface_covered"].to_frame())
#ds_prop_GBoost_test["property_surface_total"] = scaler.fit_transform(ds_prop_GBoost_test["property_surface_total"].to_frame())
#ds_prop_GBoost_test["longitud"] = scaler.fit_transform(ds_prop_GBoost_test["longitud"].to_frame())
#ds_prop_GBoost_test["latitud"] = scaler.fit_transform(ds_prop_GBoost_test["latitud"].to_frame())

standard_scaler = StandardScaler()

ds_prop_GBoost_train["property_surface_covered"] = standard_scaler.fit_transform(ds_prop_GBoost_train["property_surface_covered"].to_frame())
ds_prop_GBoost_train["property_surface_total"] = standard_scaler.fit_transform(ds_prop_GBoost_train["property_surface_total"].to_frame())
ds_prop_GBoost_train["longitud"] = standard_scaler.fit_transform(ds_prop_GBoost_train["longitud"].to_frame())
ds_prop_GBoost_train["latitud"] = standard_scaler.fit_transform(ds_prop_GBoost_train["latitud"].to_frame())
#ds_prop_XGBoost_train["property_rooms"] = standard_scaler.fit_transform(ds_prop_XGBoost_train["property_rooms"].to_frame())

ds_prop_GBoost_test["property_surface_covered"] = standard_scaler.fit_transform(ds_prop_GBoost_test["property_surface_covered"].to_frame())
ds_prop_GBoost_test["property_surface_total"] = standard_scaler.fit_transform(ds_prop_GBoost_test["property_surface_total"].to_frame())
ds_prop_GBoost_test["longitud"] = standard_scaler.fit_transform(ds_prop_GBoost_test["longitud"].to_frame())
ds_prop_GBoost_test["latitud"] = standard_scaler.fit_transform(ds_prop_GBoost_test["latitud"].to_frame())
#ds_prop_XGBoost_test["property_rooms"] = standard_scaler.fit_transform(ds_prop_XGBoost_test["property_rooms"].to_frame())


In [308]:
#Hacemos division Train-Test
features = ['property_rooms','property_surface_total','latitud','longitud', 'property_type_PH', 'property_type_Departamento']
target = ["property_price"]

x_train = ds_prop_GBoost_train.drop(columns=["property_price"])
y_train = ds_prop_GBoost_train["property_price"]

x_test = ds_prop_GBoost_test.drop(columns=["property_price"])
y_test = ds_prop_GBoost_test["property_price"]

#x_train = ds_prop_GBoost_train[features]
#x_test = ds_prop_GBoost_test[features]

#y_train = ds_prop_GBoost_train[target]
#y_test = ds_prop_GBoost_test[target]

In [309]:
# Establecemos un grid con las combinaciones de valores que queremos evaluar. Combinaciones resultantes=3^4=81
grid_hiperparametros = {'n_estimators': [50, 100], #nº de etapas de boosting
                        'learning_rate': [0.01, 0.1], #reduce la contribucion de cada arbol por este valor 
                        'max_features': [4, 5], #nº de variables a tener en cuenta para las divisiones
                        'min_samples_split': [5, 10]} #nº mínimo de observaciones necesarias para dividir un nodo interno (n.minobsinnode en R)


# Definimos las métricas de interés
metricas = {'r2':make_scorer(metrics.r2_score), 'mse': make_scorer(mean_squared_error)}

# Definimos los parámetros de búsqueda por validación cruzada
busqueda_grid = GridSearchCV(estimator = ensemble.GradientBoostingRegressor(criterion='mse', 
                                                                            loss='ls', #funcion a optimizar = least squares
                                                                            random_state=0), 
                             param_grid = grid_hiperparametros, 
                             cv = 10, #folds
                             #return_train_score = True,
                             scoring = metricas,
                             refit = "mse",
                             n_jobs = -1) 

# Aplicamos la busqueda
busqueda_grid.fit(x_train, y_train)

# Mejores hiperparametros sobre los datos hold-out y métricas de validación cruzada
print('Conjunto de hiperparámetros óptimo:\n', busqueda_grid.best_params_,
     '\nMSE +- std =', round(busqueda_grid.cv_results_['mean_test_mse'][busqueda_grid.best_index_],3),
     '+-', round(busqueda_grid.cv_results_['std_test_mse'][busqueda_grid.best_index_],3),
     '\nR^2 =', round(busqueda_grid.cv_results_['mean_test_r2'][busqueda_grid.best_index_],3),
     '+-', round(busqueda_grid.cv_results_['std_test_r2'][busqueda_grid.best_index_],3))

C:\Users\Fabiana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\ensemble\_gb.py:542: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warnings.warn(
C:\Users\Fabiana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\ensemble\_gb.py:294: FutureWarning: The loss 'ls' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  warnings.warn(
C:\Users\Fabiana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\tree\_classes.py:397: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warnings.warn(
C:\Users\Fabiana\AppDat

Conjunto de hiperparámetros óptimo:
 {'learning_rate': 0.01, 'max_features': 4, 'min_samples_split': 5, 'n_estimators': 50} 
MSE +- std = 10639170787.742 +- 695404144.867 
R^2 = 0.43 +- 0.005


C:\Users\Fabiana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\tree\_classes.py:397: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warnings.warn(
C:\Users\Fabiana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\tree\_classes.py:397: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warnings.warn(
C:\Users\Fabiana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\tree\_classes.py:397: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warnings.warn(
C:\User

In [310]:
# Ajustamos el modelo GradientBoosting usando los hiperparametros optimos obtenidos
modelo_boostingR = ensemble.GradientBoostingRegressor(criterion='mse', # criterio para la division
                                                      n_estimators = 100,
                                                      max_features = 4, 
                                                      learning_rate = 0.01,
                                                      min_samples_split = 15,
                                                      random_state = 0) # semilla
                                                     

# Entrenamos el modelo con los datos de entrenamiento 
modelo_boostingR.fit(x_train, y_train)

C:\Users\Fabiana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\ensemble\_gb.py:542: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warnings.warn(
C:\Users\Fabiana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\tree\_classes.py:397: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warnings.warn(
C:\Users\Fabiana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\tree\_classes.py:397: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warnings.warn(
C:\Users

GradientBoostingRegressor(criterion='mse', learning_rate=0.01, max_features=4,
                          min_samples_split=15, random_state=0)

In [321]:
#Mejor métrica
mse = busqueda_grid.best_score_
print("RMSE en datos de entrnamiento: " + str(np.sqrt(mse)))

RMSE en datos de entrnamiento: 105839.25276163602


In [311]:
y_pred = modelo_boostingR.predict(x_test)

In [312]:
# TEST

# Métricas en test
print('Métricas en test:',
      '\nMAE =', round(metrics.mean_absolute_error(y_test, y_pred),3),
      '\nMSE =', round(metrics.mean_squared_error(y_test, y_pred),3),
      '\nRMSE =', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred)),3),
      '\nR^2 =', round(modelo_boostingR.score(x_test, y_test),3))

Métricas en test: 
MAE = 53138.318 
MSE = 7032212083.976 
RMSE = 83858.286 
R^2 = 0.632


Explicacion de metricas:

En este modelo elegimos las metricas MAE, MSE, RMSE y R2. Siendo:
MAE -> Error absoluto medio
MSE -> Error cuadratico medio
RMSE -> Raiz del error cuadratico medio

In [313]:
#filename = "gradient_boost"
#pickle.dump(modelo_boostingR, open(filename, 'wb'))

#### Ahora usamos el dataset reducido

In [314]:
x_train = ds_prop_GBoost_reducido_train.drop(columns=["target"])
y_train = ds_prop_GBoost_reducido_train["target"]

x_test = ds_prop_GBoost_reducido_test.drop(columns=["target"])
y_test = ds_prop_GBoost_reducido_test["target"]


In [315]:
ds_prop_XGBoost_reducido_test

,cp_1,cp_2,cp_3,cp_4,cp_5,cp_6,target
0,-1.203931,-0.275293,0.413176,0.281218,-0.653813,0.322816,85000.0
1,-1.556395,0.103435,0.233243,-0.015272,0.565592,-0.188810,520000.0
2,-2.230063,-0.194028,1.526780,-0.091470,0.809760,-0.160262,87000.0
3,1.678872,0.694012,-0.608398,0.044571,-2.015136,-0.732341,690000.0
4,-1.511679,0.021077,1.780005,-0.186017,0.574209,-0.148672,95000.0
...,...,...,...,...,...,...,...
16257,-1.510756,-0.733333,-0.120271,0.570586,-1.558772,0.267688,152000.0
16258,2.940023,1.843282,0.785799,-0.677593,-0.320143,-0.514126,109000.0
16259,-0.214688,0.659277,-1.824546,0.078141,-0.005580,-0.387280,194000.0
16260,0.191963,0.957573,0.024753,-0.285457,0.633941,-0.186328,142000.0


In [316]:
ds_prop_GBoost_reducido_train

,cp_1,cp_2,cp_3,cp_4,cp_5,cp_6,target
0,-1.203931,-0.275293,0.413176,0.281218,-0.653813,0.322816,80000.0
1,-1.556395,0.103435,0.233243,-0.015272,0.565592,-0.188810,79900.0
2,-2.230063,-0.194028,1.526780,-0.091470,0.809760,-0.160262,69000.0
3,1.678872,0.694012,-0.608398,0.044571,-2.015136,-0.732341,150000.0
4,-1.511679,0.021077,1.780005,-0.186017,0.574209,-0.148672,85000.0
...,...,...,...,...,...,...,...
59846,-2.002204,0.023406,-2.141913,0.378118,0.582682,0.273304,70000.0
59847,0.129276,0.646630,1.565487,-0.334909,0.135125,-0.360703,158000.0
59848,0.377534,0.654601,-1.306694,0.147717,-0.460932,0.286666,175000.0
59849,-1.090624,0.035584,0.529826,0.098430,0.007456,0.265632,122000.0


In [317]:
# Establecemos un grid con las combinaciones de valores que queremos evaluar. Combinaciones resultantes=3^4=81
grid_hiperparametros = {'n_estimators': [50, 100], #nº de etapas de boosting
                        'learning_rate': [0.01, 0.1], #reduce la contribucion de cada arbol por este valor 
                        'max_features': [4, 5], #nº de variables a tener en cuenta para las divisiones
                        'min_samples_split': [5, 10]} #nº mínimo de observaciones necesarias para dividir un nodo interno (n.minobsinnode en R)


# Definimos las métricas de interés
metricas = {'r2':make_scorer(metrics.r2_score), 'mse': make_scorer(mean_squared_error)}

# Definimos los parámetros de búsqueda por validación cruzada
busqueda_grid = GridSearchCV(estimator = ensemble.GradientBoostingRegressor(criterion='mse', 
                                                                            loss='ls', #funcion a optimizar = least squares
                                                                            random_state=0), 
                             param_grid = grid_hiperparametros, 
                             cv = 10, #folds
                             #return_train_score = True,
                             scoring = metricas,
                             refit = "mse",
                             n_jobs = -1) 

# Aplicamos la busqueda
busqueda_grid.fit(x_train, y_train)

# Mejores hiperparametros sobre los datos hold-out y métricas de validación cruzada
print('Conjunto de hiperparámetros óptimo:\n', busqueda_grid.best_params_,
     '\nMSE +- std =', round(busqueda_grid.cv_results_['mean_test_mse'][busqueda_grid.best_index_],3),
     '+-', round(busqueda_grid.cv_results_['std_test_mse'][busqueda_grid.best_index_],3),
     '\nR^2 =', round(busqueda_grid.cv_results_['mean_test_r2'][busqueda_grid.best_index_],3),
     '+-', round(busqueda_grid.cv_results_['std_test_r2'][busqueda_grid.best_index_],3))

C:\Users\Fabiana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\ensemble\_gb.py:542: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warnings.warn(
C:\Users\Fabiana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\ensemble\_gb.py:294: FutureWarning: The loss 'ls' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  warnings.warn(
C:\Users\Fabiana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\tree\_classes.py:397: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warnings.warn(
C:\Users\Fabiana\AppDat

Conjunto de hiperparámetros óptimo:
 {'learning_rate': 0.01, 'max_features': 4, 'min_samples_split': 5, 'n_estimators': 50} 
MSE +- std = 11201947425.141 +- 721337655.199 
R^2 = 0.4 +- 0.006


C:\Users\Fabiana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\tree\_classes.py:397: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warnings.warn(
C:\Users\Fabiana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\tree\_classes.py:397: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warnings.warn(


In [318]:
# Ajustamos el modelo GradientBoosting usando los hiperparametros optimos obtenidos
modelo_boostingR = ensemble.GradientBoostingRegressor(criterion='mse', # criterio para la division
                                                      n_estimators = 100,
                                                      max_features = 4, 
                                                      learning_rate = 0.01,
                                                      min_samples_split = 15,
                                                      random_state = 0) # semilla
                                                     

# Entrenamos el modelo con los datos de entrenamiento 
modelo_boostingR.fit(x_train, y_train)

C:\Users\Fabiana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\ensemble\_gb.py:542: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warnings.warn(
C:\Users\Fabiana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\tree\_classes.py:397: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warnings.warn(
C:\Users\Fabiana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\tree\_classes.py:397: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warnings.warn(
C:\Users

GradientBoostingRegressor(criterion='mse', learning_rate=0.01, max_features=4,
                          min_samples_split=15, random_state=0)

In [319]:
y_pred = modelo_boostingR.predict(x_test)

In [320]:
# TEST

# Métricas en test
print('Métricas en test:',
      '\nMAE =', round(metrics.mean_absolute_error(y_test, y_pred),3),
      '\nMSE =', round(metrics.mean_squared_error(y_test, y_pred),3),
      '\nRMSE =', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred)),3),
      '\nR^2 =', round(modelo_boostingR.score(x_test, y_test),3))

Métricas en test: 
MAE = 103610.994 
MSE = 23635018200.042 
RMSE = 153736.847 
R^2 = -0.238


## Conclusiones

El modelo que elegiriamos para predecir el precio de la propiedad seria XGBoost, dado a su mejor performance en train y test, respecto a los dos modelos restantes, KNN y GradientBoost